In [1]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [2]:
!docker build --no-cache -t huggingface-pytorch-inference-detectron2 .

Sending build context to Docker daemon    194kB
Step 1/16 : FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-inference:1.10.2-transformers4.17.0-gpu-py38-cu113-ubuntu20.04
1.10.2-transformers4.17.0-gpu-py38-cu113-ubuntu20.04: Pulling from huggingface-pytorch-inference

17ec1767: Already exists 
45a9c0c5: Already exists 
e4c1f40f: Already exists 
86ab2510: Already exists 
adc78060: Already exists 
c6e63a52: Pulling fs layer 
97ee13cc: Pulling fs layer 
a59a00e6: Pulling fs layer 
a1e3bf84: Pulling fs layer 
3b772e88: Pulling fs layer 
43a4848f: Pulling fs layer 
006305fa: Pulling fs layer 
71df8f99: Pulling fs layer 
8a177eab: Pulling fs layer 
16bc43dc: Pulling fs layer 
18b99c95: Pulling fs layer 
a1e3bf84: Waiting fs layer 
e4a94b36: Pulling fs layer 
400d90c3: Pulling fs layer 
bd7973a6: Pulling fs layer 
b772e88: Waiting fs layer 
361aa367: Pulling fs layer 
fd2d813f: Pulling fs layer 
34432534: Pull complete 071kB/2.071kBBDownloading  655.7MB/3.149GBDownloadin

In [3]:
!ls

build-image.ipynb  Dockerfile


# Upload image to ECR

In [ ]:
%%sh

# Specify an algorithm name
algorithm_name=huggingface-pytorch-inference-detectron2

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Log into Docker
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

# docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}